In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, GridspecLayout
from IPython.display import display
from sqlalchemy import create_engine, Column, Integer, Float, String, Boolean, func, text
from sqlalchemy.orm import declarative_base, sessionmaker


In [ ]:
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    racket = Column(String)
    tension = Column(Integer)
    stringType = Column(String)
    count = Column(Integer)
    completed = Column(Integer)
    owed = Column(Float)
    paid = Column(Boolean)
    def __repr__(self):
        return f'Customer: {self.name}, Racket: {self.racket}, Tension: {self.tension}, StringType: {self.stringType}, Remaining to String: {self.count - self.completed}, Owed: {self.owed}, Paid: {self.paid}'


In [3]:
engine = create_engine('sqlite:///badminton_customer.db')
Base.metadata.create_all(engine)

def ensure_customer_columns(engine):
    with engine.connect() as conn:
        cols = [row[1] for row in conn.execute(text("PRAGMA table_info(customers)")).fetchall()]
        if 'tension' not in cols:
            conn.execute(text("ALTER TABLE customers ADD COLUMN tension INTEGER"))
        if 'stringType' not in cols:
            conn.execute(text("ALTER TABLE customers ADD COLUMN stringType VARCHAR"))
        conn.commit()

ensure_customer_columns(engine)

all_names = []
all_rackets = []

Session = sessionmaker(bind=engine)
session = Session()

name_input_label = widgets.Label("Customer Name:")

name_selection = widgets.Combobox(
    placeholder = 'Add Customer',
options = list(sorted(set(customer.name for customer in session.query(Customer).all() if customer.name))),
    ensure_option = False
)
name_button = widgets.Button(
    description = 'Confirm',
    button_style = 'success'
)


In [4]:
racket_selection = widgets.Combobox(
    placeholder = 'Racket Name',
    options = list(customer.racket for customer in session.query(Customer).all() if customer.racket),
    ensure_option = False
)

add_racket = widgets.Button(
    description = 'Add Racket',
    button_style = 'success'
)


In [ ]:
# UI widgets
name_label = widgets.Label()
racket_label = widgets.Label()
count_label = widgets.Label()
completed_label = widgets.Label()
remaining_label = widgets.Label()
owed_label = widgets.Label()
paid_mark = widgets.Label()

tension_input = widgets.BoundedIntText(value=0, min=0, max=80, description='Tension')
string_type_input = widgets.Text(value='', description='String Type', placeholder='e.g., BG65 / Aerobite')

entries_table_box = VBox([])

pending_add = {'name': None, 'racket': None, 'tension': None, 'stringType': None}
confirm_message = widgets.HTML('')
confirm_ok = widgets.Button(description='OK', button_style='success')
confirm_cancel = widgets.Button(description='Cancel', button_style='warning')
confirm_box = VBox([confirm_message, HBox([confirm_ok, confirm_cancel])])
confirm_box.layout.display = 'none'
confirm_box.layout.border = '1px solid #ccc'
confirm_box.layout.padding = '10px'
confirm_box.layout.margin = '8px 0 8px 0'

show_unpaid_button = widgets.Button(description='Show Unpaid Customers', button_style='info')
unpaid_table_box = VBox([])
unpaid_table_box.layout.display = 'none'

delete_all_button = widgets.Button(description='Delete All Records', button_style='danger')


In [9]:
current = {'customer': None}


In [10]:
def refresh_options():
    global all_names, all_rackets
    all_names = [c[0] for c in session.query(Customer.name).filter(Customer.name.isnot(None)).all() if c[0]]
    name_selection.options = list(sorted(set(all_names)))

    all_rackets = [r[0] for r in session.query(Customer.racket).filter(Customer.racket.isnot(None)).all() if r[0]]
    racket_selection.options = list(sorted(set(all_rackets)))


In [11]:
def show_customer(customer):
    unpaid_table_box.layout.display = 'none'
    if not customer or not customer.name:
        entries_table_box.children = tuple()
        return

    name_key = customer.name.lower()
    entries = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == name_key)
        .filter(Customer.racket.isnot(None))
        .filter(Customer.racket != '')
        .order_by(Customer.id.asc())
        .all()
    )

    name_label.value = f'Name: {customer.name.title() if customer.name else ""}'
    racket_label.value = ''
    count_label.value = f'Total Entries: {len(entries)}'
    completed_label.value = ''
    remaining_label.value = ''

    unpaid_count = sum(1 for e in entries if not bool(e.paid))
    total_owed = float(unpaid_count * 20)
    owed_label.value = f'Total Owed: ${total_owed}'
    paid_mark.value = 'Paid: ✅' if total_owed == 0.0 else 'Paid: ❌'

    n_rows = len(entries) + 1
    grid = GridspecLayout(n_rows, 6, grid_gap='6px')
    headers = ['Name', 'Racket', 'Tension', 'String Type', 'Paid', 'Delete']
    col_widths = ['140px', '220px', '80px', '220px', '70px', '90px']

    for col, h in enumerate(headers):
        grid[0, col] = widgets.HTML(f"<b>{h}</b>")
        grid[0, col].layout = widgets.Layout(width=col_widths[col])

    for i, row in enumerate(entries, start=1):
        paid_cb = widgets.Checkbox(value=bool(row.paid), indent=False, layout=widgets.Layout(width=col_widths[4]))
        del_btn = widgets.Button(description='Delete', button_style='danger', layout=widgets.Layout(width=col_widths[5]))

        def make_on_paid(entry_id):
            def _on_change(change):
                if change.get('name') != 'value':
                    return
                rec = session.query(Customer).filter_by(id=entry_id).first()
                if not rec:
                    return
                rec.paid = bool(change.get('new'))
                session.commit()
                show_customer(customer)
            return _on_change

        def make_on_delete(entry_id):
            def _on_click(_b):
                rec = session.query(Customer).filter_by(id=entry_id).first()
                if not rec:
                    return
                session.delete(rec)
                session.commit()
                refresh_options()
                show_customer(customer)
            return _on_click

        paid_cb.observe(make_on_paid(row.id), names='value')
        del_btn.on_click(make_on_delete(row.id))

        grid[i, 0] = widgets.Label(row.name or '', layout=widgets.Layout(width=col_widths[0]))
        grid[i, 1] = widgets.Label(row.racket or '', layout=widgets.Layout(width=col_widths[1]))
        grid[i, 2] = widgets.Label('' if row.tension is None else str(row.tension), layout=widgets.Layout(width=col_widths[2]))
        grid[i, 3] = widgets.Label(row.stringType or '', layout=widgets.Layout(width=col_widths[3]))
        grid[i, 4] = paid_cb
        grid[i, 5] = del_btn

    entries_table_box.children = (grid,)


In [12]:
details_box = VBox([
    HBox([racket_selection, tension_input, string_type_input, add_racket]),
    confirm_box,
    HBox([show_unpaid_button, delete_all_button]),
    VBox([name_label, racket_label, count_label, completed_label, remaining_label, owed_label, paid_mark]),
    entries_table_box,
    unpaid_table_box,
])
name_button.layout.display = "none"
name_row = HBox([name_input_label, name_selection, name_button])

root = VBox([name_row])


In [13]:
def on_name_typing(change):
    typed = change['new'] or ''
    # Don't strip here - preserve what user is typing for dropdown
    if typed:
        typed_lower = typed.lower()
        # Case-insensitive filtering - prioritize matches that start with typed text
        starts_with = [n for n in all_names if n.lower().startswith(typed_lower)]
        contains = [n for n in all_names if typed_lower in n.lower() and n not in starts_with]
        # Combine: starts_with first, then contains, then the typed value
        filtered = starts_with + contains
        # Always include the typed value so combobox can accept it
        if typed not in filtered:
            filtered.append(typed)
        name_selection.options = list(filtered)  # Use list instead of tuple
    else:
        name_selection.options = list(sorted(set(all_names)))  # Use list instead of tuple
    
    if current['customer'] is None:
        name_button.layout.display = 'none' if typed == '' else 'inline-flex'
    else:
        name_button.layout.display = 'none'


In [14]:
def on_racket_typing(change):
    typed = change['new'] or ''
    # Don't strip here - preserve what user is typing for dropdown
    if typed:
        typed_lower = typed.lower()
        # Case-insensitive filtering - prioritize matches that start with typed text
        starts_with = [r for r in all_rackets if r.lower().startswith(typed_lower)]
        contains = [r for r in all_rackets if typed_lower in r.lower() and r not in starts_with]
        # Combine: starts_with first, then contains, then the typed value
        filtered = starts_with + contains
        # Always include the typed value so combobox can accept it
        if typed not in filtered:
            filtered.append(typed)
        racket_selection.options = list(filtered)  # Use list instead of tuple
    else:
        racket_selection.options = list(sorted(set(all_rackets)))  # Use list instead of tuple


In [15]:
def confirm_name(b):
    name = name_selection.value.strip()
    if name.lower() in {'add customer', 'customer name'}:
        return
    if not name:
        return

    name_title = name.title()
    existing = session.query(Customer).filter(func.lower(Customer.name) == name.lower()).first()

    if not existing:
        # Create a 'profile' row (no racket). Entries are separate rows with racket set.
        existing = Customer(
            name=name_title,
            racket=None,
            tension=None,
            stringType=None,
            count=0,
            completed=0,
            owed=0.0,
            paid=False,
        )
        session.add(existing)
        session.commit()
        refresh_options()
    else:
        # normalize stored casing
        if existing.name != name_title:
            existing.name = name_title
            session.commit()
            refresh_options()

    current['customer'] = existing
    show_customer(existing)
    if details_box not in root.children:
        root.children = (name_row, details_box)


In [18]:
def on_racket_change(b):
    if current['customer'] is None or not current['customer'].name:
        # Auto-confirm/create customer if user typed a name but didn't click Confirm
        confirm_name(None)
        if current['customer'] is None or not current['customer'].name:
            return

    racket = racket_selection.value.strip()
    if not racket:
        return

    tension = tension_input.value
    string_type = (string_type_input.value or '').strip()

    racket_title = racket.title()

    # Enforce uniqueness: (name, racket, tension, stringType)
    exists = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == current['customer'].name.lower())
        .filter(func.lower(Customer.racket) == racket_title.lower())
        .filter(Customer.tension == tension)
        .filter(func.coalesce(Customer.stringType, '') == string_type)
        .first()
    )
    if exists:
        # Duplicate - do nothing
        racket_selection.value = ''
        return

    entry = Customer(
        name=current['customer'].name,
        racket=racket_title,
        tension=tension,
        stringType=string_type,
        count=1,
        completed=0,
        owed=0.0,
        paid=False,
    )
    session.add(entry)
    session.commit()

    refresh_options()
    show_customer(current['customer'])

    racket_selection.value = ''
    tension_input.value = 0
    string_type_input.value = ''


In [20]:
def ensure_selected_customer():
    c = current.get('customer')
    if c is None or not getattr(c, 'name', None):
        return None
    # If user changed the dropdown but didn't Confirm, do not switch customers implicitly
    return c


In [ ]:
# Event wiring
name_selection.observe(on_name_typing, names='value')
name_button.on_click(confirm_name)
racket_selection.observe(on_racket_typing, names='value')

# Ensure Confirm starts hidden until user types/selects a name
name_button.layout.display = 'none'

# Delete all records (two-click confirm)
delete_all_confirm = {'armed': False}

def on_delete_all(_b):
    c = current.get('customer')
    if c is None or not getattr(c, 'name', None):
        return
    if not delete_all_confirm['armed']:
        delete_all_confirm['armed'] = True
        delete_all_button.description = 'Confirm Delete All'
        return
    delete_all_confirm['armed'] = False
    delete_all_button.description = 'Delete All Records'
    name_key = c.name.lower()
    entries = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == name_key)
        .filter(Customer.racket.isnot(None))
        .filter(Customer.racket != '')
        .all()
    )
    for e in entries:
        session.delete(e)
    session.commit()
    refresh_options()
    show_customer(c)

delete_all_button.on_click(on_delete_all)

# Add racket confirmation flow

def begin_add_racket_confirmation(_b):
    c = ensure_selected_customer()
    if c is None:
        return
    racket = (racket_selection.value or '').strip()
    if not racket:
        return

    pending_add['name'] = c.name
    pending_add['racket'] = racket.title()
    pending_add['tension'] = tension_input.value
    pending_add['stringType'] = (string_type_input.value or '').strip()

    confirm_message.value = (
        "<b>Confirm Add</b><br>"
        "Customer: <b>{}</b><br>"
        "Racket: <b>{}</b><br>"
        "Tension: <b>{}</b><br>"
        "String Type: <b>{}</b>"
    ).format(
        pending_add['name'],
        pending_add['racket'],
        pending_add['tension'],
        pending_add['stringType'],
    )
    confirm_box.layout.display = 'block'


def confirm_add_ok(_b):
    if not pending_add.get('name') or not pending_add.get('racket'):
        return
    exists = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == pending_add['name'].lower())
        .filter(func.lower(Customer.racket) == pending_add['racket'].lower())
        .filter(Customer.tension == pending_add['tension'])
        .filter(func.coalesce(Customer.stringType, '') == pending_add['stringType'])
        .first()
    )
    if exists:
        confirm_box.layout.display = 'none'
        return

    entry = Customer(
        name=pending_add['name'],
        racket=pending_add['racket'],
        tension=pending_add['tension'],
        stringType=pending_add['stringType'],
        count=1,
        completed=0,
        owed=0.0,
        paid=False,
    )
    session.add(entry)
    session.commit()

    confirm_box.layout.display = 'none'
    refresh_options()

    c = current.get('customer')
    if c is not None:
        show_customer(c)

    racket_selection.value = ''
    tension_input.value = 0
    string_type_input.value = ''
    pending_add['name'] = None
    pending_add['racket'] = None


def confirm_add_cancel(_b):
    confirm_box.layout.display = 'none'
    pending_add['name'] = None
    pending_add['racket'] = None


confirm_ok.on_click(confirm_add_ok)
confirm_cancel.on_click(confirm_add_cancel)
add_racket.on_click(begin_add_racket_confirmation)

# Unpaid customers view

def show_unpaid_customers(_b):
    unpaid_table_box.layout.display = 'block'
    confirm_box.layout.display = 'none'

    rows = (
        session.query(Customer.name)
        .filter(Customer.racket.isnot(None))
        .filter(Customer.racket != '')
        .filter((Customer.paid == False) | (Customer.paid.is_(None)))
        .all()
    )
    names = sorted(set(r[0] for r in rows if r[0]))

    n_rows = len(names) + 1
    grid = GridspecLayout(n_rows, 3, grid_gap='6px')
    headers = ['Customer', 'Unpaid Count', 'View']
    widths = ['200px', '120px', '90px']

    for col, h in enumerate(headers):
        grid[0, col] = widgets.HTML(f'<b>{h}</b>')
        grid[0, col].layout = Layout(width=widths[col])

    for i, name in enumerate(names, start=1):
        unpaid_count = (
            session.query(Customer)
            .filter(func.lower(Customer.name) == name.lower())
            .filter(Customer.racket.isnot(None))
            .filter(Customer.racket != '')
            .filter((Customer.paid == False) | (Customer.paid.is_(None)))
            .count()
        )

        view_btn = widgets.Button(description='View', button_style='primary', layout=Layout(width=widths[2]))

        def make_view(nm):
            def _click(_x):
                name_selection.value = nm
                confirm_name(None)
                unpaid_table_box.layout.display = 'none'
            return _click

        view_btn.on_click(make_view(name))

        grid[i, 0] = widgets.Label(name, layout=Layout(width=widths[0]))
        grid[i, 1] = widgets.Label(str(unpaid_count), layout=Layout(width=widths[1]))
        grid[i, 2] = view_btn

    unpaid_table_box.children = (grid,)


show_unpaid_button.on_click(show_unpaid_customers)


In [21]:
refresh_options()
display(root)
